## Hello Everyone !!
In this Notebook we will be using EfficientNet B3 for Leaf Disease Classification. Most parts of this notebook is adapted version of [Alex Shonenkov](https://www.kaggle.com/shonenkov)'s work.

# Import Dependecies

In [ ]:
!pip install -q efficientnet_pytorch > /dev/null 

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.optim.optimizer import Optimizer
import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import time
from datetime import datetime
from tqdm.autonotebook import tqdm
from efficientnet_pytorch import EfficientNet

import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

# Fold Splitting

I have made splitted the data in 5 folds. Each fold has same number of 5 different classes. The csv can be found [here](https://www.kaggle.com/zaber666/cld-dataset).

In [ ]:
df = pd.read_csv('/kaggle/input/cld-dataset/train_5fold.csv')

In [ ]:
#samples per fold
df.fold.value_counts()

In [ ]:
#different labels per fold
df_fold1 = df[df.fold == 1]
df_fold1.label.value_counts()

# Augmentation

In [ ]:
def train_transform():
    return A.Compose(
        [
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Rotate(),
            A.Resize(height=256, width=256, p=1),
            A.Cutout(num_holes=8, max_h_size=32, max_w_size=32, fill_value=0, p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.0
    )

def valid_transform():
    return A.Compose(
        [
            A.Resize(height=256, width=256, p=1),
            ToTensorV2(p=1.0),
        ], p=1.0
    )

# Dataset & Dataloader

I have used resized 256x256 images from [here](https://www.kaggle.com/konradb/resized-data-256).

In [ ]:
DATA_DIR = '/kaggle/input/resized-data-256/train_images'

In [ ]:
class CLD_Dataset(Dataset):
    def __init__(self, image_ids, labels, transform=None):
        super().__init__()
        self.image_ids = image_ids
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        image_id = self.image_ids[index]
        label = self.labels[index]

        image = cv2.imread(f'{DATA_DIR}/{image_id}',  cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = image / 255.0

        if self.transform is not None:
            image = self.transform(**{'image':image})['image']

        ohe_label = torch.zeros(5, dtype=torch.float32)
        ohe_label[label] = 1

        return image, ohe_label
    
    def __len__(self):
        return len(self.image_ids)

In [ ]:
FOLD = 0
DEBUG = False

if not DEBUG:
    dataset_train = CLD_Dataset(
        image_ids = df[df.fold != FOLD].image_id.values,
        labels = df[df.fold != FOLD].label.values,
        transform = train_transform()
    )
    dataset_val = CLD_Dataset(
        image_ids = df[df.fold == FOLD].image_id.values,
        labels = df[df.fold == FOLD].label.values,
        transform = valid_transform()
    )
else:
    dataset_train = CLD_Dataset(
        image_ids = df[df.fold != FOLD].image_id.values[:128],
        labels = df[df.fold != FOLD].label.values[:128],
        transform = train_transform()
    )
    dataset_val = CLD_Dataset(
        image_ids = df[df.fold == FOLD].image_id.values[:128],
        labels = df[df.fold == FOLD].label.values[:128],
        transform = valid_transform()
    ) 

In [ ]:
figure, axes = plt.subplots(nrows=3, ncols=2, figsize=(10,10))
for i in range(6):
    image, _ = dataset_train[i]
    image = image.permute(1,2,0).cpu().numpy()
    axes[i//2, i%2].imshow(image)

In [ ]:
BATCH_SIZE = 32

In [ ]:
train_loader = DataLoader(
    dataset_train,
    batch_size=BATCH_SIZE,
    drop_last=True,
    num_workers=2,
    shuffle=True
)
val_loader = DataLoader(
    dataset_val,
    batch_size=BATCH_SIZE,
    drop_last=False,
    num_workers=2
)

<b>We will mixup in this notebook. Let's see some exmaples of mixup</b>

In [ ]:
def mixup(data, targets, alpha):
    
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lm = np.random.beta(alpha, alpha)
    data = lm*data + (1-lm)*shuffled_data
    
    targets = (targets, shuffled_targets, lm)

    return data, targets

In [ ]:
images, targets = next(iter(train_loader))
images, _ = mixup(images, targets, 0.5) 

In [ ]:
figure, axes = plt.subplots(nrows=3, ncols=2, figsize=(10,10))
for i in range(6):
    image = images[i]
    image = image.permute(1,2,0).cpu().numpy()
    axes[i//2, i%2].imshow(image)

# Model

In [ ]:
def make_model():
    model = EfficientNet.from_pretrained('efficientnet-b4') 
    model._fc = nn.Linear(in_features=1792, out_features=5, bias=True)
    return model

In [ ]:
model = make_model()

# Training

In [ ]:
class LabelSmoothing(nn.Module):
    def __init__(self, smoothing = 0.01):
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing

    def forward(self, x, target):
        if self.training:
            x = x.float()
            target = target.float()
            logprobs = torch.nn.functional.log_softmax(x, dim = -1)

            nll_loss = -logprobs * target
            nll_loss = nll_loss.sum(-1)
    
            smooth_loss = -logprobs.mean(dim=-1)

            loss = self.confidence * nll_loss + self.smoothing * smooth_loss

            return loss.mean()
        else:
            return torch.nn.functional.cross_entropy(x, target)

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class Fitter:
    def __init__(self, model, device, work_dir, epochs):
        self.epoch = 0
        self.model = model
        self.work_dir = work_dir
        if not os.path.exists(self.work_dir):
            os.makedirs(self.work_dir)
        self.log_path = f'{self.work_dir}/log.txt'
        self.best_score = 0
        self.device = device
        self.model.to(device)

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.0005)
        self.scheduler = torch.optim.lr_scheduler.OneCycleLR(
                                    self.optimizer, 
                                    max_lr=0.001, 
                                    epochs=30,  
                                    steps_per_epoch=int(len(dataset_train) / BATCH_SIZE), 
                                    pct_start=0.1, 
                                    anneal_strategy='cos', 
                                    final_div_factor=10**5)
        
        self.criterion = LabelSmoothing().to(self.device)

        self.log(f'Fitter prepared. Device is {self.device}')

    def log(self, message):
        with open(self.log_path, 'a+') as f:
            f.write(f'{message}\n')
        print(message)

    def fit(self, train_loader, val_loader):
        for epc in range(20):
            lr = self.optimizer.param_groups[0]['lr']
            timestamp = datetime.utcnow().isoformat()
            self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            loss, accuracy = self.train_one_epoch(train_loader)
            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, loss: {loss.avg:.5f}, accuracy: {accuracy.avg:.5f}, time: {(time.time() - t):.5f}')

            t = time.time()
            loss, accuracy = self.validation(val_loader)
            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, loss: {loss.avg:.5f}, accuracy: {accuracy.avg:.5f}, time: {(time.time() - t):.5f}')

            if accuracy.avg > self.best_score:
                self.best_score = accuracy.avg
                self.save_model(f'{self.work_dir}/best-checkpoint.bin')
            
            self.save_model(f'{self.work_dir}/last-checkpoint.bin')
            self.epoch += 1

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        accuracy = AverageMeter()

        t = time.time()
        tk = tqdm(train_loader, total=len(train_loader), desc='Training')
        for step, (images, targets) in enumerate(tk):

            targets = targets.to(self.device).float()
            images = images.to(self.device).float()
            batch_size = images.shape[0]

            if np.random.random() < 0.5 :
                # mixup
                
                mixup_images, mixup_targets = mixup(images, targets, 0.4)
                targets, shuffled_targets, lm = mixup_targets
                self.optimizer.zero_grad()
                outputs = self.model(mixup_images)
                
                loss = lm*self.criterion(outputs, targets) + (1-lm)*self.criterion(outputs, shuffled_targets)
            else:
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, targets)

            loss.backward()
            outputs = nn.functional.softmax(outputs, dim=1)
            acc = (outputs.argmax(1)==targets.argmax(1)).sum().item() / batch_size

            summary_loss.update(loss.detach().item(), batch_size)
            accuracy.update(acc, batch_size)
            self.optimizer.step()


        self.scheduler.step()
        return summary_loss, accuracy

    def validation(self, val_loader):
        self.model.eval()
        summary_loss = AverageMeter()
        accuracy = AverageMeter()

        t = time.time()
        tk = tqdm(val_loader, total=len(val_loader), desc='Validating')
        for step, (images, targets) in enumerate(tk):
                
            with torch.no_grad():
                targets = targets.to(self.device).float()
                images = images.to(self.device).float()
                batch_size = images.shape[0]
                outputs = self.model(images)
                loss = self.criterion(outputs, targets)
                outputs = nn.functional.softmax(outputs, dim=1)
                acc = (outputs.argmax(dim=1)==targets.argmax(dim=1)).sum().item() / batch_size
                summary_loss.update(loss.detach().item(), batch_size)
                accuracy.update(acc, batch_size)

        return summary_loss, accuracy
    
    def save_model(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict':self.model.state_dict(),
            'optimizer_state_dict':self.optimizer.state_dict(),
            'scheduler_state_dict':self.scheduler.state_dict(),
            'best_score':self.best_score,
            'epoch':self.epoch
        }, path)

    def load_model(self, path):
        ckpt = torch.load(path, map_location=self.device)
        self.model.load_state_dict(ckpt['model_state_dict'])
        self.optimizer.load_state_dict(ckpt['optimizer_state_dict'])
        self.scheduler.load_state_dict(ckpt['scheduler_state_duct'])
        self.best_score = ckpt['best_score']
        self.epoch = ckpt['epoch']

In [ ]:
fitter = Fitter(model=model, device='cuda', work_dir='/kaggle/working/output', epochs=30)

In [ ]:
#fitter.fit(train_loader, val_loader) 

Have fun changing different parts and experimenting.

Happy Coding.